# Evaluationsmethoden für LLM-Outputs

In diesem Notebook lernen wir drei wichtige Methoden zur Evaluation und Qualitätssicherung von LLM-Outputs kennen:

1. **LLM-As-A-Judge**: Wie ein LLM zur Bewertung von LLM-Antworten verwendet werden kann
2. **NLP-basierte Testkriterien**: Objektive Metriken zur Bewertung von Textqualität
3. **PI Scrubbing** mit Microsoft Presidio: Entfernung personenbezogener Daten für Datenschutz

In [ ]:
# Notwendige Imports
from helpers import llm
from langchain.prompts import ChatPromptTemplate
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain.schema import StrOutputParser
import pandas as pd

## 1. LLM-As-A-Judge

Bei dieser Methode verwenden wir ein LLM, um die Ausgaben eines anderen LLM zu bewerten. Das LLM fungiert als "Richter" und evaluiert die Antworten nach bestimmten Kriterien.

### Prinzipien des LLM-As-A-Judge

- **Objektivität**: Das LLM wird angewiesen, klare Bewertungskriterien anzuwenden
- **Konsistenz**: Die Bewertung sollte über verschiedene Antworten hinweg konsistent sein
- **Mehrfachkriterien**: Die Bewertung kann nach verschiedenen Dimensionen erfolgen (Korrektheit, Relevanz, Sprachqualität, etc.)

### Implementierung mit LangChain

In [ ]:
# Beispielanfrage und verschiedene Antworten
frage = "Wie wirkt sich die Nutzung erneuerbarer Energien auf die Wirtschaft aus?"
antworten = [
    "Erneuerbare Energien schaffen Arbeitsplätze und reduzieren Umweltverschmutzung. Die Abhängigkeit von fossilen Brennstoffen wird verringert.",
    "Die Verwendung von erneuerbaren Energien hat großartige Auswirkungen auf die Wirtschaft! Einfach toll!",
    "Erneuerbare Energien haben positive und negative wirtschaftliche Auswirkungen. Positiv: neue Arbeitsplätze, Innovationen, Reduktion von Importabhängigkeiten. Negativ: hohe Anfangsinvestitionen, mögliche Destabilisierung bestehender Energiemarktstrukturen und Übergangsprobleme für Arbeitskräfte in traditionellen Energiesektoren."
]

In [ ]:
# Verwendung des integrierten Evaluators von LangChain
# Wir wollen die Antworten auf Relevanz bewerten lassen
evaluator = load_evaluator(EvaluatorType.CRITERIA, criteria="relevance", llm=llm())

results = []
for i, antwort in enumerate(antworten):
    result = evaluator.evaluate_strings(prediction=antwort, input=frage)
    results.append({
        "Antwort ID": i+1,
        "Bewertung": result["score"],
        "Begründung": result["reasoning"]
    })

# Ergebnisse als Tabelle anzeigen
pd.DataFrame(results)

### Mehrere Kriterien gleichzeitig bewerten

In [ ]:
# Mehrere Kriterien gleichzeitig bewerten
multi_criteria_evaluator = load_evaluator(
    EvaluatorType.CRITERIA, 
    criteria={
        "Relevanz": "Ist die Antwort relevant zur gestellten Frage?",
        "Vollständigkeit": "Enthält die Antwort ausreichend Informationen zur Beantwortung der Frage?",
        "Sachlichkeit": "Ist die Antwort sachlich und nicht durch Emotionen oder Meinungen geprägt?"
    },
    llm=llm()
)

multi_results = []
for i, antwort in enumerate(antworten):
    result = multi_criteria_evaluator.evaluate_strings(prediction=antwort, input=frage)
    result_dict = {
        "Antwort ID": i+1,
    }
    # Extrahiere alle Bewertungen
    for key, value in result["criteria"].items():
        result_dict[f"{key}"] = value["score"] 
        result_dict[f"{key} (Begründung)"] = value["reasoning"]
    
    multi_results.append(result_dict)

# Ergebnisse als Tabelle anzeigen
pd.DataFrame(multi_results)

### Benutzerdefinierte LLM-als-Richter Implementierung

Wir können auch einen eigenen Ansatz implementieren, der spezifischer auf unsere Anforderungen zugeschnitten ist:

In [ ]:
# Benutzerdefiniertes Judge-Prompt
judge_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Du bist ein erfahrener Experte für Textqualität. Deine Aufgabe ist es, die Qualität von Antworten zu bewerten.
    
    Bewerte die Antwort nach den folgenden Kriterien auf einer Skala von 1-10:
    1. Faktengenauigkeit (1-10): Sind die präsentierten Informationen korrekt?
    2. Vollständigkeit (1-10): Werden alle wichtigen Aspekte der Frage behandelt?
    3. Objektivität (1-10): Ist die Antwort ausgewogen und frei von tendenziösen Aussagen?
    4. Verständlichkeit (1-10): Ist die Antwort klar und leicht zu verstehen?
    
    Strukturiere deine Antwort wie folgt:
    ```json
    {
        "faktengenauigkeit": {WERT},
        "faktengenauigkeit_begründung": "{BEGRÜNDUNG}",
        "vollständigkeit": {WERT},
        "vollständigkeit_begründung": "{BEGRÜNDUNG}",
        "objektivität": {WERT},
        "objektivität_begründung": "{BEGRÜNDUNG}",
        "verständlichkeit": {WERT},
        "verständlichkeit_begründung": "{BEGRÜNDUNG}",
        "gesamtpunktzahl": {WERT},
        "zusammenfassung": "{KURZE ZUSAMMENFASSUNG}"
    }
    ```
    
    Die Gesamtpunktzahl ist der Durchschnitt der vier Einzelbewertungen.
    """),
    ("human", """Bitte bewerte die folgende Antwort auf die Frage:
    
    Frage: {frage}
    
    Antwort: {antwort}""")
])

# Strukturierte Ausgabe für unser LLM
class EvaluationResult:
    faktengenauigkeit: int
    faktengenauigkeit_begründung: str
    vollständigkeit: int
    vollständigkeit_begründung: str
    objektivität: int
    objektivität_begründung: str
    verständlichkeit: int
    verständlichkeit_begründung: str
    gesamtpunktzahl: float
    zusammenfassung: str

# Chain aufbauen mit strukturierter Ausgabe
evaluation_chain = judge_prompt | llm().with_structured_output(EvaluationResult)

# Ergebnisse sammeln
custom_results = []
for i, antwort in enumerate(antworten):
    result = evaluation_chain.invoke({"frage": frage, "antwort": antwort})
    custom_results.append({
        "Antwort ID": i+1,
        "Faktengenauigkeit": result.faktengenauigkeit,
        "Vollständigkeit": result.vollständigkeit,
        "Objektivität": result.objektivität,
        "Verständlichkeit": result.verständlichkeit,
        "Gesamtpunktzahl": result.gesamtpunktzahl,
        "Zusammenfassung": result.zusammenfassung
    })

# Ergebnisse als Tabelle anzeigen
pd.DataFrame(custom_results)

## 2. NLP-basierte Testkriterien

Neben dem LLM-As-A-Judge-Ansatz können auch objektive, NLP-basierte Metriken zur Bewertung von Texten herangezogen werden. Diese sind weniger subjektiv als LLM-Bewertungen und können automatisiert werden.

### Beispiel: Textmetriken mit NLTK und spaCy

In [ ]:
# Installation der benötigten Bibliotheken (falls nötig)
# %pip install nltk spacy
# %pip install textstat

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import textstat

# NLTK-Daten herunterladen (falls noch nicht geschehen)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    
def calculate_nlp_metrics(text):
    """Berechnet verschiedene NLP-Metriken für einen Text"""
    # Grundlegende Metriken
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    
    # Lesbarkeit (Flesch Reading Ease)
    readability = textstat.flesch_reading_ease(text)
    
    # Durchschnittliche Satzlänge
    avg_sentence_length = len(words) / len(sentences) if len(sentences) > 0 else 0
    
    # Fog Index (Gunning Fog) - Misst die Lesbarkeit des Textes
    fog_index = textstat.gunning_fog(text)
    
    # Automatisierte Lesbarkeitsindex (estimiert die Schuljahre, die zum Verständnis notwendig sind)
    automated_readability = textstat.automated_readability_index(text)
    
    return {
        "Anzahl Wörter": len(words),
        "Anzahl Sätze": len(sentences),
        "Durchschnittliche Satzlänge": round(avg_sentence_length, 2),
        "Flesch Reading Ease": round(readability, 2),
        "Gunning Fog Index": round(fog_index, 2),
        "Automated Readability Index": round(automated_readability, 2)
    }

# Berechnung für alle Antworten
nlp_metrics = []
for i, antwort in enumerate(antworten):
    metrics = calculate_nlp_metrics(antwort)
    metrics["Antwort ID"] = i+1
    nlp_metrics.append(metrics)

# Ergebnisse als Tabelle anzeigen
pd.DataFrame(nlp_metrics)

### Interpretation der NLP-Metriken

- **Flesch Reading Ease**: Höhere Werte bedeuten leichter zu lesen (100 = sehr einfach, 0-30 = akademisch/komplex)
- **Gunning Fog Index**: Gibt an, wie viele Jahre formaler Bildung benötigt werden, um den Text zu verstehen
- **Automated Readability Index**: Ähnlich wie Gunning Fog, aber mit anderer Formel berechnet

### Bewertung mit Textsimilarität

Ein weiteres wichtiges Kriterium ist die semantische Ähnlichkeit zwischen einer Musterantwort und der zu bewertenden Antwort.

In [ ]:
# Beispiel einer Musterantwort
musterantwort = """
Die Nutzung erneuerbarer Energien hat diverse wirtschaftliche Auswirkungen: Sie schafft neue Arbeitsplätze im grünen Energiesektor, reduziert langfristig Energiekosten und verringert die Abhängigkeit von Energieimporten. Zudem fördert sie Innovationen und neue Technologien. Allerdings erfordert der Umstieg hohe Anfangsinvestitionen und kann zu wirtschaftlichen Anpassungsproblemen in traditionellen Energiesektoren führen. Langfristig überwiegen jedoch die wirtschaftlichen Vorteile durch Kosteneinsparungen, geringere Umweltbelastungen und neue Wachstumsbranchen.
"""

# Semantic Textual Similarity mit Embeddings
from langchain.embeddings import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Embeddings Modell laden
embeddings = OpenAIEmbeddings()

# Embedding der Musterantwort berechnen
muster_embedding = embeddings.embed_query(musterantwort)

# Embeddings für jede Antwort berechnen und Ähnlichkeit bestimmen
similarity_results = []
for i, antwort in enumerate(antworten):
    antwort_embedding = embeddings.embed_query(antwort)
    # Cosinus-Ähnlichkeit berechnen (zwischen 0 und 1, höher ist ähnlicher)
    similarity = cosine_similarity(
        np.array(muster_embedding).reshape(1, -1), 
        np.array(antwort_embedding).reshape(1, -1)
    )[0][0]
    
    similarity_results.append({
        "Antwort ID": i+1,
        "Semantische Ähnlichkeit": round(similarity, 4),
        "Ähnlichkeit in %": round(similarity * 100, 2)
    })

# Ergebnisse als Tabelle anzeigen
pd.DataFrame(similarity_results)

## 3. PI Scrubbing mit Microsoft Presidio

Personenbezogene Informationen (PI) müssen oft aus Texten entfernt werden, bevor diese für Trainingsdaten oder zur Evaluation verwendet werden können. Microsoft Presidio ist ein Open-Source-Tool zur Erkennung und Anonymisierung von personenbezogenen Daten.

### Installation von Presidio

In [ ]:
# Installation von Presidio (falls nötig)
# %pip install presidio-analyzer presidio-anonymizer spacy
# %python -m spacy download de_core_news_lg

In [ ]:
# Presidio für PII-Erkennung und Anonymisierung
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import RecognizerResult, OperatorConfig

# Beispieltext mit personenbezogenen Daten
text_mit_pii = """
Sehr geehrter Herr Müller,

vielen Dank für Ihre Anfrage. Wie gewünscht sende ich Ihnen die Informationen zu unserem Projekt. 
Bitte kontaktieren Sie mich unter meiner E-Mail max.schmidt@example.com oder telefonisch unter +49 176 12345678.

Ihr Kundenkonto mit der Nummer DE987654321 wurde aktualisiert. Weitere Details finden Sie in der beigefügten Rechnung.

Die Lieferung erfolgt an Ihre Adresse in der Musterstraße 123, 10115 Berlin.

Mit freundlichen Grüßen,
Dr. Anna Weber
Geburtsdatum: 15.04.1982
"""

# Analyzer und Anonymizer initialisieren
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

# Text analysieren und PII erkennen
# Wir verwenden die deutsche Sprache
results = analyzer.analyze(text=text_mit_pii, language="de")

# Erkannte PII-Elemente anzeigen
print("Erkannte personenbezogene Daten:")
for result in results:
    print(f"- {result.entity_type}: {text_mit_pii[result.start:result.end]} (Konfidenz: {result.score:.2f})")

# Text anonymisieren
anonymized_text = anonymizer.anonymize(
    text=text_mit_pii,
    analyzer_results=results
)

print("\nAnonymisierter Text:")
print(anonymized_text.text)

### Anonymisierungsoptionen in Presidio

Presidio bietet verschiedene Methoden zur Anonymisierung:

In [ ]:
from presidio_anonymizer.entities import OperatorConfig

# Verschiedene Anonymisierungsstrategien demonstrieren
operators = {
    "PERSON": OperatorConfig("redact"),  # Komplett entfernen
    "PHONE_NUMBER": OperatorConfig("mask", {"masking_char": "*", "chars_to_mask": 10}),  # Maskieren
    "EMAIL_ADDRESS": OperatorConfig("replace", {"new_value": "[EMAIL]"}),  # Ersetzen
    "LOCATION": OperatorConfig("hash"),  # Hash-Wert
    "DATE_TIME": OperatorConfig("encrypt", {"key": "my-key"})  # Verschlüsseln
}

# Text mit benutzerdefinierten Operatoren anonymisieren
anonymized_text_custom = anonymizer.anonymize(
    text=text_mit_pii,
    analyzer_results=results,
    operators=operators
)

print("Anonymisierter Text mit benutzerdefinierten Strategien:")
print(anonymized_text_custom.text)

### Integration in LLM-Workflows

Die Anonymisierung kann als Vorverarbeitungsschritt in LLM-Workflows integriert werden:

In [ ]:
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer

# Langchain-Integration mit dem experimentellen Modul
anonymizer = PresidioReversibleAnonymizer()

# Text anonymisieren
anonymized_result = anonymizer.anonymize(text_mit_pii)
anonymized_text = anonymized_result["text"]

print("Anonymisierter Text mit LangChain:")
print(anonymized_text)

# Nun können wir diesen anonymisierten Text an ein LLM senden
response = llm().invoke(f"Fasse den folgenden Text zusammen: {anonymized_text}")

print("\nLLM-Antwort auf anonymisierten Text:")
print(response.content)

# Bei Bedarf kann der Text de-anonymisiert werden
deanonymized_text = anonymizer.deanonymize(anonymized_text)

print("\nDe-anonymisierter Text:")
print(deanonymized_text)

## Zusammenfassung und Best Practices

Die drei vorgestellten Methoden sind komplementär und können je nach Anwendungsfall kombiniert werden:

1. **LLM-As-A-Judge**:
   - Vorteile: Flexibel, kontextbezogen, kann komplexe Aspekte bewerten
   - Nachteile: Subjektiv, nicht vollständig deterministisch, kostenintensiv
   - Anwendungsfälle: Bewertung von Kreativität, Genauigkeit komplexer Informationen

2. **NLP-basierte Testkriterien**:
   - Vorteile: Objektiv, reproduzierbar, kostengünstig
   - Nachteile: Begrenzte semantische Tiefe, keine Bewertung von Kontext
   - Anwendungsfälle: Lesbarkeitsanalyse, stilistische Konsistenz, formale Anforderungen

3. **PI Scrubbing mit Presidio**:
   - Vorteile: DSGVO-konform, automatisierbar, anpassbar
   - Nachteile: Kann Kontextinformationen entfernen, false positives möglich
   - Anwendungsfälle: Anonymisierung von Trainingsdaten, Compliance-Anforderungen

### Best Practices für die Evaluation von LLM-Outputs:

1. **Mehrere Methoden kombinieren** für eine umfassende Bewertung
2. **Menschliche Überprüfung** für kritische Anwendungen einbeziehen
3. **Domänenspezifische Kriterien** definieren je nach Anwendungsfall
4. **Kontinuierliche Evaluation** während der Entwicklung durchführen
5. **Feedback-Schleifen etablieren** zur Verbesserung des Systems